# 연구 목표(목적)
---

*   텐서플로우를 이용하여 예측 결과까지의 흐름을 확인 
*   절차적으로 어떤것들이 나오는지 간략하게 확인 
*   데이터는 BMI 지수 = 몸무게 / (키*키)
*   비만도 측정한다






# 데이터 획득 
---

In [0]:
import numpy as np 
import pandas as pd

In [0]:
path = f'/content/drive/My Drive/Colab Notebooks/dl_data/{name}.{data_type}'
name = '3_bmi'
data_type = 'csv'
# path = f'./dl_data/{name}.{data_type}'

df = pd.read_csv(path)
df.head()

In [0]:
# 데이터 확인 
df.shape 

In [0]:
# 컬럼 확인 
df.columns

In [0]:
# 레이블 컬럼의 줄복값 제외한 본값을 출력 

df.label.unique()

# 데이터 준비 
--- 
- 품질향상 

In [0]:
# 값의 편차가 크다 .
# 키의 최대값, 몸무게의 최대값을 기준 => 정규화 처리
# 기존 컬럼값을 정규화하여 대체할것 

df.height.min(), df.height.max() 

#  최소는 200
#  최대는 200 임을 확인 

In [0]:
# 정규화 
test = df['weight'] / df['weight'].max()
test

In [0]:
# 왜 ? => 큰 의미는 없음 -> 그냥 최대 값 기준 
df['weight'] = df['weight'] / df['weight'].max()
df['height'] = df['height'] / df['height'].max()

In [0]:
# 
df.head(2)

## label 값을 원-핫인코딩 응용으로 처리
---

- 딥러닝시 데이터는 모두 수치여야 함
- 레이블을 0과 1로 표현-> 원핫인코딩
- [1,0,0] or [0,1,0], or [0,0,1]으로 대체
- 레이블이 1칸으로 표현이 된다
- 'thin'   : [1,0,0]
- 'normal' : [0,1,0]
- 'fat'    : [0,0,1] <- ndarray 형으로 처리

In [0]:
#  사본 생성 
df2 = df[:]

In [0]:
# 사전화 -> 간소화 (더미값 없이)
mapping = {
'thin'   : np.array([1,0,0]),
'normal' : np.array([0,1,0]),
'fat'    : np.array([0,0,1])
}

In [0]:
df2['label']

In [0]:
 # 대입한다 mapping에 요소 값을 대입-> 키값으로 리턴 받는다.
 df2['label'].apply(lambda x : mapping[x])

In [0]:
# df['label']값이 위에 언급된 배열로 세팅되겠금 처리 
df2['label'] =  df2['label'].apply(lambda x : mapping[x])

In [0]:
# 데이터 분리 
# 데이터를 훈련용, 테스트용 75:25로 분류
from sklearn.model_selection import train_test_split

In [0]:
# X,y를 분리
X_train,X_test, y_train,y_test = train_test_split(
    df2[df2.columns[:2]], df['label'], random_state = 0
)

In [0]:
X_train.shape, X_test.shape

# 데이터 분석(생략)

# 데이터 모델링(딥러닝, 텐서플로우 엔진사용)

# 구성
---

- 현재 데이터는 n개의 입력에 대해서, thin, normal, fat등의 3개의 결론으로 도달
- 신경망 구성을 위해서는 퍼셉트론을 적용해 본다면, n개에 입력(키, 몸무게, 15000, 5000)에 가중치를 곱하고, 편향(바이어스) 더한다(값을 올리거나, 내러거나) 
    - 이렇게 만들어진 값을 모두 합하여, 활성화함수(각각의 특성)를 통과 시키면 결과에 도착하게 된다(출력층에 도달하게 된다)
    - 이런 신경망을 여러개 구성할수도 있지만 여기서는 1개만 적용
    - 활성화 함수는 softmax()를 사용
    - 입력대비 출력에 대한 식은  X*W + b라는 퍼셉트론을 기반으로 구성하겠다.

## softmax 활성화 함수
---

- 입력받은 값을 출력으로 0~1사이로 모두 정규화하여서 출력값들의 총 합은 1이 되게 하는 함수
- softmax 함수의 출력값은 범주형 확률분포라고 표현하고
- 통상, 다중 클래스들을 예측할때, 출력층에 도달하기 직전에 사용한다. output activation으로 사용한다

In [0]:
from IPython.display import Image
name = 'sm0'
path = f'/content/drive/My Drive/Colab Notebooks/dl_data/{name}.png'
Image(path, width=500 )

In [0]:
from IPython.display import Image
name = 'sm1'
path = f'/content/drive/My Drive/Colab Notebooks/dl_data/{name}.png'
Image(path, width=500 )

In [0]:
from IPython.display import Image
name = 'sm2'
path = f'/content/drive/My Drive/Colab Notebooks/dl_data/{name}.png'
Image(path, width=500 )

In [0]:
from IPython.display import Image
name = 'sm3'
path = f'/content/drive/My Drive/Colab Notebooks/dl_data/{name}.png'
Image(path, width=500 )

1. X의 shape을 결정    

2. W의 shape가 결정된다 (행렬의 곱은 교환법칙이 특수한 케이스 빼고는 성립이 않되므로)

3. b의 shape을 결정

4. y에 도달하기 위해서 활성화함수를 사용

5. 필요하면 행렬을 플래툰하는 작업도 나올수는 있다 y에 가기 위해서(여기서는 제외)


## 1부. 데이터 플로우 그래프 생성
---
- **y = softmax(xW + b)**

In [0]:
# 텐서플로우 불러오기 
import tensorflow as tf
%tensorflow_version 1.x

In [0]:
# 플레이스 홀더 
# x는 데이터가 주입되는 것 
# 데이터가 몇개 주입될지 모르겠으나, 각 데이터의 최종 성분은 2개이다.
# None 어떤 값이라도 들어 갈수있다 그대시 차원은 맞춰야 한다 
x = tf. placeholder( tf.float32, [None, 2]) 
# 정수 값이 들어 간다.

In [0]:
# 가중치 - 
# 값이 바뀔 수 있다. -> 변수라는것을 의미 
# W의 shape >>> ( 2, 3 ) 

W = tf.Variable(tf.zeros([2,3]))


In [0]:
# 편향 , 바이어스 => 브로드 캐스팅에 의해 자동 증가 된다(shape)
# 차원을 자동으로 맞춰져서 연산이 문제없데 수행되도록 지원
# b의 shape(3) = >  y에 맞춰서 계산 

b = tf.Variable(tf.zeros([3]))


In [0]:
# y는 데이터 플로우 그래프, 계산식 
# y = softmax(xW+b)

y = tf.nn.softmax(tf.matmul(x, W)+b)


In [0]:
(x.shape, W.shape, b.shape, y.shape)

In [0]:
'%s%s + %s = %s' % (x.shape, W.shape, b.shape, y.shape)

## 2부. 실행
---

- 최적화 기법이 가미
  - 좋은 모델에 대한 정의:기준점
    - 비용(cost), 손실(loss)등 값들이 원하는 값(임계치, 목표치)에서 얼마나 떨어져 있는가? 이것을 판단의 지표로 본다
    - 줄이는 방향으로 간다
    - 크로스 엔트로피와 같은 알고리즘을 적용 한다
    - 크로스 엔트로피:정보압축알고리즘, 배팅, 머신러닝등등에서 주로 사용

In [0]:
# y_ : 정답 레이블 => shape => [None, 3]
# y  : 예측 레이블
y_ = tf.placeholder( tf.float32, [None, 3] )

In [0]:
# 크로스 엔트로피 데이터 플로우 그래프
# reduce_sum( 여기에 들어오는 행렬(tensor)들의 구성원들의 총합 )
cross_entropy = - tf.reduce_sum( y_*tf.log(y) )

In [0]:
from IPython.display import Image
name = 'sm4'
img_type = 'png'
# path = f'./dl_data/{name}.{data_type}'
path = f'/content/drive/My Drive/Colab Notebooks/dl_data/{name}.{img_type}'
Image(path, width=500 )



In [0]:
# 크로스 엔트로피가 작동하여 최소화로 결과가 수렵되도록 작업이 진행 
# 이 진행을 담당하는 알고리즘 => 경사하강법(Gradient Descent alagorithm)
# 텐서플로우 엔진에서 변수를 비용을 줄어드는 방향으로 조금씩 이동 
# 이를 통해서 연산이 수행 -> 비용, 손실값이 최소가 되도록 처리한다. 


hyper_param = 0.01 
optimazer = tf.train.GradientDescentOptimizer(hyper_param)


In [0]:
# 크로스 엔트로피와 경사하강법 연결
# minimize(), 손실값을 최소화하겠금 변수를 조금씩 이동시키는 함수
train = optimazer.minimize( cross_entropy )

In [0]:
# 예측
# 결과, thin, normal, fat에 대한 각각의 % 비율중 가장 큰값을 예측값으로 판단
# 이런 데이터 중에 최대값 => argmax
predict = tf.equal( tf.argmax( y, 1 ), tf.argmax( y_, 1 ) )

In [0]:
# 정확도
accuracy = tf.reduce_mean( tf.cast( predict, tf.float32 ) )

## 실행
---

- Session 획득
- 훈련데이터, 정답, 테스트 데이터등등을 주입하여 결과를 확인

In [0]:
# 한번 훈련시킬때 사용하는 데이터의 양 
train_term = 100

# tf 구동 

# 0. tf의 세션을 연다 
with tf.Session() as sess:
    # 1. tf의 모든 변수를 초기화 

    init_val = tf.global_variables_initializer()
    sess.run(init_val) 

    # 2. 훈련 방법을 고려 

    #       1만5천개를 한번에 넣을것인가? 혹은 구간을 나누어서 사용할 것인가
    #        양이 중요한가? 횟수가 중요한가? 횟수로 총 양을 맞출 수도 있다. 
    # 이런 데이터의 양을 가지고 몇번 훈련 시킬것인가 
    train_count = int(X_train.shape[0]/ train_term)  # float => int로 변경 
    for step in range(train_count):
        # X_train으로부터 이번 차수에 훈련할 데이터를 추출 
        # 훈련 회수 증가 이유 => 경사 하강법에 의해 손실값이 이동되는 것을 확인하기 위함 
        offset = step * train_term  # offset만 끔 띄워가면서 데이터를 추출 

    # 3. 이번 차시에 훈련시킬 데이터를 획득
        cur_train_data = X_train[ offset : offset + train_term ] # [0: 0 + 100]
    
    # 4. 학습 
        # 플로우에 주입할 데이터를 명시 
        # -> 플로우 상에 존재하는 모든 플레이스 홀더(placeholer)를 기술한다.
        dic = {
            x  : cur_train_data,    # x에 들어 가는 훈련용데이터(100,2)   
            y_ : list(y_train[ offset : offset + train_term ]) # 리스트 형식의 y_train[0:0+100]까지의 사본 
        }
        sess.run( train, feed_dict=dic ) # feed_dict를 train에 대입(투입,할당) 시킨다.
    
    # 5. 중간 평가 
        if step % 10 == 0:
            # 로그 출력 
            # 크로스 엔트로피값 획득 
            cross_en = sess.run( cross_entropy, feed_dict=dic )
            # 정확도 확득 => 훈련시 사용하지 않는 데이터 사용 
            # 중간 점검이므로, 학습에 영향을 미치지 않는다. 
            # X_test, y_test
            testDic = {
                x:X_test,
                y_:list( y_test )}
            acc =   sess.run( accuracy, feed_dict=testDic )

            # 출력 
            print( f'단계={step} 손실값={cross_en} 정확도={acc}' )
    # 훈련완료
    testDic = {
        x:X_test,
        y_:list( y_test )
        }
    acc      = sess.run( accuracy, feed_dict=testDic )
    print('정확도', acc)



In [0]:
a = X_train.shape[0]/100
type(a)

In [0]:
print(list(y_train[ 0 : 0 + 100 ]))

### 상승
---

- 훈련회수를 더욱 증가시켜서
- 한번 훈련시 데이터는 100개는 동일


In [0]:
type(y_train)

In [0]:
# from IPython.display import Image
# name = 'sm1'
# path = f'/content/drive/My Drive/Colab Notebooks/dl_data/{name}.png'
# Image(path, width=500 )